In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.ticker as mtick
import sqlite3
import seaborn as sns
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import requests   
import shutil      
import datetime
from scipy.stats import norm
import os
import winsound

home_folder = 'C:\\Users\\Travis\\OneDrive\\Data Science\\Personal_Projects\\Sports\\NBA_Prediction_V3_1'
os.chdir(home_folder)

In [2]:
def replace_name_values(filename):
        # replace values with dashes for compatibility
    filename = filename.replace('%','_')
    filename = filename.replace('=','_')
    filename = filename.replace('?','_')
    filename = filename.replace('&','_')
    filename = filename.replace('20Season_','')
    filename = filename.replace('20Season','')
    filename = filename.replace('_SeasonType','')
    filename = filename.replace('SeasonYear_','')
    filename = filename.replace('Season_','')
    filename = filename.replace('Regular Season','Regular_Season')
    return filename

In [3]:
def get_data_w_double_header(url_list, file_folder, option_numbers,option_names, headers_to_skip): 
        i = 0
        optionz = np.arange(0,option_numbers, 1)
        # get first option
        for u in url_list:
            for option in optionz:
                driver.get(u)
                time.sleep(1)
                # get option xpath
                op = option + 1

                try:
                        xpath_option = '/html/body/div[1]/div[2]/div[2]/div[3]/section[1]/div/div/div[4]/label/div/select/option[' + str(op) + ']' # click option
                        elem = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, xpath_option))).click()
                except:
                        print(f'{u} did not load. Moving to next url.')
                        continue
                
                elem = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, xpath_option)))
                driver.find_element(by=By.XPATH, value=xpath_option).click()
                
                # all pages
                xpath_all = '//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[3]/div/label/div/select/option[1]' 
                elem = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, xpath_all)))
                driver.find_element(by=By.XPATH, value=xpath_all).click()
                src = driver.page_source
                parser = BeautifulSoup(src, "lxml")
                table = parser.find("table", attrs = {"class":"Crom_table__p1iZz"})
                headers = table.findAll('th')
                headerlist = [h.text.strip() for h in headers[headers_to_skip:]]  # THIS IS THE DOUBLE HEADER THAT WE ARE SKIPPING
                row_names = table.findAll('a')                             # find rows
                row_list = [b.text.strip() for b in row_names[0:]] 
                rows = table.findAll('tr')[0:] 
                player_stats = [[td.getText().strip() for td in rows[i].findAll('td')[0:]] for i in range(len(rows))]
                tot_cols = len(player_stats[2])                           #set the length to ignore hidden columns
                headerlist = headerlist[:tot_cols]   
                stats = pd.DataFrame(player_stats, columns = headerlist)
                filename = file_folder + str(u[34:]).replace('/', '_') + 'op_' + str(option_names[option]) + '.csv'
                filename = replace_name_values(filename)
                pd.DataFrame.to_csv(stats, filename)
                i += 1
                lu = len(url_list)
                print(f'{filename} Completed Successfully! {i} / {lu} Complete!')

In [4]:
def get_shot_dash_doubleheader(url_list, file_folder, option_numbers,option_names, headers_to_skip): 
        i = 0
        optionz = np.arange(0,option_numbers, 1)
        # get first option
        for u in url_list:
            for option in optionz:
                driver.get(u)
                time.sleep(1)
                # get option xpath
                op = option + 1

                try:
                        xpath_option = '//*[@id="__next"]/div[2]/div[2]/div[3]/section[1]/div/div/div[4]/label/div/select/option[' + str(op) + ']'
                        elem = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, xpath_option)))
                        driver.find_element(by=By.XPATH, value=xpath_option).click()
                except:
                        print(f'{u} did not load. Moving to next url.')
                        continue
                
                
                # all pages
                xpath_all = '//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[3]/div/label/div/select/option[1]' 
                elem = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, xpath_all)))
                driver.find_element(by=By.XPATH, value=xpath_all).click()
                src = driver.page_source
                parser = BeautifulSoup(src, "lxml")
                table = parser.find("table", attrs = {"class":"Crom_table__p1iZz"})
                headers = table.findAll('th')
                headerlist = [h.text.strip() for h in headers[headers_to_skip:]]  # THIS IS THE DOUBLE HEADER THAT WE ARE SKIPPING
                row_names = table.findAll('a')                             # find rows
                row_list = [b.text.strip() for b in row_names[0:]] 
                rows = table.findAll('tr')[0:] 
                player_stats = [[td.getText().strip() for td in rows[i].findAll('td')[0:]] for i in range(len(rows))]
                tot_cols = len(player_stats[2])                           #set the length to ignore hidden columns
                headerlist = headerlist[:tot_cols]   
                stats = pd.DataFrame(player_stats, columns = headerlist)
                filename = file_folder + str(u[34:]).replace('/', '_') + 'op_' + str(option_names[option]) + '.csv'
                filename = replace_name_values(filename)
                pd.DataFrame.to_csv(stats, filename)
                i += 1
                lu = len(url_list)
                print(f'{filename} Completed Successfully! {i} / {lu} Complete!')

In [5]:
def trans_urls(url):
    new_url = str(url)[34:].replace('/', '_')
    filename = replace_name_values(new_url)
    return filename

In [6]:
def append_the_data(folder, data_prefix, filename_selector):
    # Appending data together via folder and/or file name

    path = folder
    p = os.listdir(path)
    pf = pd.DataFrame(p)


    # filter for files that contain the filename_selector
    pf_reg = pf.loc[pf[0].astype(str).str.contains(filename_selector)] 

    appended_data = []
    for file in pf_reg[0]:
        data = pd.read_csv(folder + '/' + file)
        # if "Season" a column, drop it
        if 'Season' in data.columns:
            data = data.drop(columns = ['Season'])
        
        data['season'] = file[(file.find('20')):(file.find('20'))+4]
        data['season_type'] = np.where('Regular' in file, 'Regular', 'Playoffs')
        # add prefix to columns
        data = data.add_prefix(data_prefix)
        data.columns = data.columns.str.lower()
        appended_data.append(data)
    
    appended_data = pd.concat(appended_data)
    return appended_data

## General

In [7]:
player_general = 'https://www.nba.com/stats/players/shots-general/'      # 4 options - Overall, Catch&Shoot, Pullups, Less than 10 feet
player_shotclock = 'https://www.nba.com/stats/players/shots-shotclock/'  # 6 options - 24-22, 22-18, 18-15, 18-15, 15-7, 7-4, 4-0
player_dribbles = 'https://www.nba.com/stats/players/shots-dribbles/'    # 5 options - 0-drib, 1-drib, 2-drib, 3-6-drib, 7-drib
player_touchtime = 'https://www.nba.com/stats/players/shots-touch-time/' # 3 options - 0-2, 2-6, 6+ seconds
player_closest_defender = 'https://www.nba.com/stats/players/shots-closest-defender/' # 4 options - 0-2, 2-4, 4-6, 6+ feet
player_closest_defender_plus10 = 'https://www.nba.com/stats/players/shots-closest-defender-10/' # 4 options - 0-2, 2-4, 4-6, 6+ feet

In [8]:
def get_urls():
    gen_urls = []
    years =['2021-22', '2020-21', '2019-20', '2018-19', '2017-18', '2016-17', '2015-16','2014-15', '2013-14']
    season_types = ['Regular Season', 'Playoffs']
    for year in years:
        for s_types in season_types:
            url = player_general + '?SeasonYear=' + year + '&SeasonType=' + s_types + '&Season=' + year
            gen_urls.append(str(url))

    to_download = pd.DataFrame(gen_urls, columns = ['urls'])

        # create new column with filename
    to_download['filename1'] = to_download.apply(lambda row: trans_urls(row['urls']), axis=1)
    to_download['filename_o1'] = to_download['filename1'] + 'op_Overall.csv'
    to_download['filename_o2'] = to_download['filename1'] + 'op_Catch_Shoot.csv'
    to_download['filename_o3'] = to_download['filename1'] + 'op_Pullups.csv'
    to_download['filename_o4'] = to_download['filename1'] + 'op_Less_than_10_feet.csv'
    
    return to_download

In [9]:
to_download = get_urls()

In [10]:
# Check to see if files exist already
folder1 = 'data/player/shot_dashboard/general/playoffs/'
folder1_files = os.listdir(folder1)
folder2 = 'data/player/shot_dashboard/general/regular_season/'
folder2_files = os.listdir(folder2)
files = folder1_files + folder2_files
files = [f for f in files if f.endswith('.csv')]

# to_download is names_to_check minus files
to_download1 = to_download[~to_download['filename_o1'].isin(files)]
to_download2 = to_download[~to_download['filename_o2'].isin(files)]
to_download3 = to_download[~to_download['filename_o3'].isin(files)]
to_download4 = to_download[~to_download['filename_o4'].isin(files)]

total_to_download = len(to_download1) + len(to_download2) + len(to_download3) + len(to_download4)
print('total to download: ', total_to_download)

total to download:  0


In [11]:
to_download = to_download['urls'].tolist()

# download files

if total_to_download >0:
    print('to download: ', total_to_download)
    driver = webdriver.Chrome()
    get_shot_dash_doubleheader(to_download, 'data/player/shot_dashboard/general/', 4, ['Overall', 'Catch&Shoot', 'Pullups', 'Less_than_10_feet'], 6)
    # move files if necessary

    shotdash_files = os.listdir('data/player/shot_dashboard/general/')
    for file in shotdash_files:
        if '.csv' in file:
            # move to correct folder
            if 'Regular' in file:
                shutil.move('data/player/shot_dashboard/general/' + file, 'data/player/shot_dashboard/general/regular_season/')
            else:
                shutil.move('data/player/shot_dashboard/general/' + file, 'data/player/shot_dashboard/general/playoffs/')
else:
    print('All files already downloaded')

All files already downloaded


### Update This Year

In [12]:
url = 'https://www.nba.com/stats/players/shots-general/?SeasonYear=2022-23&SeasonType=Regular%20Season&Season=2022-23'
driver = webdriver.Chrome()
get_shot_dash_doubleheader([url], 'data/player/shot_dashboard/general/', 4, ['Overall', 'Catch&Shoot', 'Pullups', 'Less_than_10_feet'], 6)

shotdash_files = os.listdir('data/player/shot_dashboard/general/')
for file in shotdash_files:
    if '.csv' in file:
        # move to correct folder
        if 'Regular' in file:
            shutil.move('data/player/shot_dashboard/general/' + file, 'data/player/shot_dashboard/general/regular_season/')
        else:
            shutil.move('data/player/shot_dashboard/general/' + file, 'data/player/shot_dashboard/general/playoffs/')

data/player/shot_dashboard/general/shots-general__2022-23_Regular_2022-23op_Overall.csv Completed Successfully! 1 / 1 Complete!
data/player/shot_dashboard/general/shots-general__2022-23_Regular_2022-23op_Catch_Shoot.csv Completed Successfully! 2 / 1 Complete!
data/player/shot_dashboard/general/shots-general__2022-23_Regular_2022-23op_Pullups.csv Completed Successfully! 3 / 1 Complete!
data/player/shot_dashboard/general/shots-general__2022-23_Regular_2022-23op_Less_than_10_feet.csv Completed Successfully! 4 / 1 Complete!


### Append

In [13]:
gen_reg_1 = append_the_data('data/player/shot_dashboard/general/regular_season/', 'sd_general_Overall__', 'op_Overall')
gen_reg_2 = append_the_data('data/player/shot_dashboard/general/regular_season/', 'sd_general_Catch_Shoot__', 'op_Catch_Shoot')
gen_reg_3 = append_the_data('data/player/shot_dashboard/general/regular_season/', 'sd_general_Pullups__', 'op_Pullups')
gen_reg_4 = append_the_data('data/player/shot_dashboard/general/regular_season/', 'sd_general_Less10_Ft__', 'op_Less_than_10_feet')

gen_play_1 = append_the_data('data/player/shot_dashboard/general/playoffs/', 'sd_general_Overall__', 'op_Overall')
gen_play_2 = append_the_data('data/player/shot_dashboard/general/playoffs/', 'sd_general_Catch_Shoot__', 'op_Catch_Shoot')
gen_play_3 = append_the_data('data/player/shot_dashboard/general/playoffs/', 'sd_general_Pullups__', 'op_Pullups')
gen_play_4 = append_the_data('data/player/shot_dashboard/general/playoffs/', 'sd_general_Less10_Ft__', 'op_Less_than_10_feet')


In [14]:
all_gen_reg1 = pd.merge(gen_reg_1, gen_reg_2,
                        left_on= ['sd_general_overall__player', 'sd_general_overall__team', 
                                    'sd_general_overall__season', 'sd_general_overall__season_type'],
                        right_on= ['sd_general_catch_shoot__player', 'sd_general_catch_shoot__team',
                                    'sd_general_catch_shoot__season', 'sd_general_catch_shoot__season_type'],
                        how = 'left')

all_gen_reg2 = pd.merge(all_gen_reg1, gen_reg_3,
                        left_on= ['sd_general_overall__player', 'sd_general_overall__team', 
                                    'sd_general_overall__season', 'sd_general_overall__season_type'],
                        right_on= ['sd_general_pullups__player', 'sd_general_pullups__team',
                                    'sd_general_pullups__season', 'sd_general_pullups__season_type'],
                        how = 'left')

all_gen_reg = pd.merge(all_gen_reg2, gen_reg_4,
                        left_on= ['sd_general_overall__player', 'sd_general_overall__team',
                                    'sd_general_overall__season', 'sd_general_overall__season_type'],
                        right_on= ['sd_general_less10_ft__player', 'sd_general_less10_ft__team',
                                    'sd_general_less10_ft__season', 'sd_general_less10_ft__season_type'],
                        how = 'left')

all_gen_play1 = pd.merge(gen_play_1, gen_play_2,
                        left_on= ['sd_general_overall__player', 'sd_general_overall__team',
                                    'sd_general_overall__season', 'sd_general_overall__season_type'],
                        right_on= ['sd_general_catch_shoot__player', 'sd_general_catch_shoot__team',
                                    'sd_general_catch_shoot__season', 'sd_general_catch_shoot__season_type'],
                        how = 'left')

all_gen_play2 = pd.merge(all_gen_play1, gen_play_3,
                        left_on= ['sd_general_overall__player', 'sd_general_overall__team',
                                    'sd_general_overall__season', 'sd_general_overall__season_type'],
                        right_on= ['sd_general_pullups__player', 'sd_general_pullups__team',
                                    'sd_general_pullups__season', 'sd_general_pullups__season_type'],
                        how = 'left')

all_gen_play = pd.merge(all_gen_play2, gen_play_4,
                        left_on= ['sd_general_overall__player', 'sd_general_overall__team',
                                    'sd_general_overall__season', 'sd_general_overall__season_type'],
                        right_on= ['sd_general_less10_ft__player', 'sd_general_less10_ft__team',
                                    'sd_general_less10_ft__season', 'sd_general_less10_ft__season_type'],
                        how = 'left')

# concat all gen reg and play
all_general = pd.concat([all_gen_reg, all_gen_play], axis = 0)

# save to csv
all_general.to_csv('data/player/aggregates/ShotDash_All_General.csv', index = False)



## Shotclock

In [15]:
def get_shotclock_urls():
    shotclock_urls = []
    years =['2021-22', '2020-21', '2019-20', '2018-19', '2017-18', '2016-17', '2015-16','2014-15', '2013-14']
    season_types = ['Regular Season', 'Playoffs']
    for year in years:
        for s_types in season_types:
            url = player_shotclock + '?SeasonYear=' + year + '&SeasonType=' + s_types + '&Season=' + year
            shotclock_urls.append(str(url))

    to_download = pd.DataFrame(shotclock_urls, columns = ['urls'])

        # create new column with filename
    to_download['filename1'] = to_download.apply(lambda row: trans_urls(row['urls']), axis=1)
    to_download['filename_o1'] = to_download['filename1'] + 'op_24-22.csv'
    to_download['filename_o2'] = to_download['filename1'] + 'op_22-18.csv'
    to_download['filename_o3'] = to_download['filename1'] + 'op_18-15.csv'
    to_download['filename_o4'] = to_download['filename1'] + 'op_15-7.csv'
    to_download['filename_o5'] = to_download['filename1'] + 'op_7-4.csv'
    to_download['filename_o6'] = to_download['filename1'] + 'op_4-0.csv'
    
    return to_download

In [16]:
# check to see if files exist already
folder1 = 'data/player/shot_dashboard/shotclock/playoffs/'
folder1_files = os.listdir(folder1)
folder2 = 'data/player/shot_dashboard/shotclock/regular_season/'
folder2_files = os.listdir(folder2)
files = folder1_files + folder2_files
files = [f for f in files if f.endswith('.csv')]

to_download = get_shotclock_urls()

# to_download is names_to_check minus files
to_download = to_download[~to_download['filename_o1'].isin(files)]
to_download = to_download[~to_download['filename_o2'].isin(files)]
to_download = to_download[~to_download['filename_o3'].isin(files)]
to_download = to_download[~to_download['filename_o4'].isin(files)]
to_download = to_download[~to_download['filename_o5'].isin(files)]
to_download = to_download[~to_download['filename_o6'].isin(files)]

to_download

,urls,filename1,filename_o1,filename_o2,filename_o3,filename_o4,filename_o5,filename_o6


In [17]:
if len(to_download) > 0:
    to_dl = to_download['urls'].tolist()
    driver = webdriver.Chrome()
    get_data_w_double_header(to_dl, 'data/player/shot_dashboard/shotclock/', 6, ['24-22', '22-18', '18-15', '15-7', '7-4', '4-0'], 6)
else:
    print('No new files to download')


No new files to download


### Update This Year

In [19]:

s_type = 'Regular Season'

url = player_shotclock + '?SeasonYear=' + '2022-23' + '&SeasonType=' + s_type + '&Season=2022-23' 

driver = webdriver.Chrome()
get_data_w_double_header([url], 'data/player/shot_dashboard/shotclock/', 6, ['24-22', '22-18', '18-15', '15-7', '7-4', '4-0'], 6)

data/player/shot_dashboard/shotclock/shots-shotclock__2022-23_Regular 2022-23op_24-22.csv Completed Successfully! 1 / 1 Complete!
data/player/shot_dashboard/shotclock/shots-shotclock__2022-23_Regular 2022-23op_22-18.csv Completed Successfully! 2 / 1 Complete!
data/player/shot_dashboard/shotclock/shots-shotclock__2022-23_Regular 2022-23op_18-15.csv Completed Successfully! 3 / 1 Complete!
data/player/shot_dashboard/shotclock/shots-shotclock__2022-23_Regular 2022-23op_15-7.csv Completed Successfully! 4 / 1 Complete!
data/player/shot_dashboard/shotclock/shots-shotclock__2022-23_Regular 2022-23op_7-4.csv Completed Successfully! 5 / 1 Complete!
data/player/shot_dashboard/shotclock/shots-shotclock__2022-23_Regular 2022-23op_4-0.csv Completed Successfully! 6 / 1 Complete!


In [20]:
# move files if necessary

shotclock_files = os.listdir('data/player/shot_dashboard/shotclock/')
for file in shotclock_files:
    if '.csv' in file:
        # move to correct folder
        if 'Regular' in file:
            shutil.move('data/player/shot_dashboard/shotclock/' + file, 'data/player/shot_dashboard/shotclock/regular_season/')
        else:
            shutil.move('data/player/shot_dashboard/shotclock/' + file, 'data/player/shot_dashboard/shotclock/playoffs/')

### Append

In [21]:
shotclock_1_reg = append_the_data('data/player/shot_dashboard/shotclock/regular_season/', 'sd_shotclock_24-22__', '24-22')
shotclock_2_reg = append_the_data('data/player/shot_dashboard/shotclock/regular_season/', 'sd_shotclock_22-18__', '22-18')
shotclock_3_reg = append_the_data('data/player/shot_dashboard/shotclock/regular_season/', 'sd_shotclock_18-15__', '18-15')
shotclock_4_reg = append_the_data('data/player/shot_dashboard/shotclock/regular_season/', 'sd_shotclock_15-7__', '15-7')
shotclock_5_reg = append_the_data('data/player/shot_dashboard/shotclock/regular_season/', 'sd_shotclock_7-4__', '7-4')
shotclock_6_reg = append_the_data('data/player/shot_dashboard/shotclock/regular_season/', 'sd_shotclock_4-0__', '4-0')

shotclock_1_play = append_the_data('data/player/shot_dashboard/shotclock/playoffs/', 'sd_shotclock_24-22__', '24-22')
shotclock_2_play = append_the_data('data/player/shot_dashboard/shotclock/playoffs/', 'sd_shotclock_22-18__', '22-18')
shotclock_3_play = append_the_data('data/player/shot_dashboard/shotclock/playoffs/', 'sd_shotclock_18-15__', '18-15')
shotclock_4_play = append_the_data('data/player/shot_dashboard/shotclock/playoffs/', 'sd_shotclock_15-7__', '15-7')
shotclock_5_play = append_the_data('data/player/shot_dashboard/shotclock/playoffs/', 'sd_shotclock_7-4__', '7-4')
shotclock_6_play = append_the_data('data/player/shot_dashboard/shotclock/playoffs/', 'sd_shotclock_4-0__', '4-0')

In [22]:
all_shotclock_reg1 = pd.merge(shotclock_1_reg, shotclock_2_reg,
                                left_on=['sd_shotclock_24-22__player', 'sd_shotclock_24-22__team', 
                                            'sd_shotclock_24-22__season', 'sd_shotclock_24-22__season_type'],
                                right_on=['sd_shotclock_22-18__player', 'sd_shotclock_22-18__team',
                                            'sd_shotclock_22-18__season', 'sd_shotclock_22-18__season_type'],
                                how='left')

all_shotclock_reg2 = pd.merge(all_shotclock_reg1, shotclock_3_reg,
                                left_on=['sd_shotclock_24-22__player', 'sd_shotclock_24-22__team',
                                            'sd_shotclock_24-22__season', 'sd_shotclock_24-22__season_type'],
                                right_on=['sd_shotclock_18-15__player', 'sd_shotclock_18-15__team',
                                            'sd_shotclock_18-15__season', 'sd_shotclock_18-15__season_type'],
                                how='left')

all_shotclock_reg3 = pd.merge(all_shotclock_reg2, shotclock_4_reg,
                                left_on=['sd_shotclock_24-22__player', 'sd_shotclock_24-22__team',
                                            'sd_shotclock_24-22__season', 'sd_shotclock_24-22__season_type'],
                                right_on=['sd_shotclock_15-7__player', 'sd_shotclock_15-7__team',
                                            'sd_shotclock_15-7__season', 'sd_shotclock_15-7__season_type'],
                                how='left')

all_shotclock_reg4 = pd.merge(all_shotclock_reg3, shotclock_5_reg,
                                left_on=['sd_shotclock_24-22__player', 'sd_shotclock_24-22__team',
                                            'sd_shotclock_24-22__season', 'sd_shotclock_24-22__season_type'],
                                right_on=['sd_shotclock_7-4__player', 'sd_shotclock_7-4__team',
                                            'sd_shotclock_7-4__season', 'sd_shotclock_7-4__season_type'],
                                how='left')

all_shotclock_reg = pd.merge(all_shotclock_reg4, shotclock_6_reg,
                                left_on=['sd_shotclock_24-22__player', 'sd_shotclock_24-22__team',
                                            'sd_shotclock_24-22__season', 'sd_shotclock_24-22__season_type'],
                                right_on=['sd_shotclock_4-0__player', 'sd_shotclock_4-0__team',
                                            'sd_shotclock_4-0__season', 'sd_shotclock_4-0__season_type'],
                                how='left')

all_shotclock_play1 = pd.merge(shotclock_1_play, shotclock_2_play,
                                left_on=['sd_shotclock_24-22__player', 'sd_shotclock_24-22__team',
                                            'sd_shotclock_24-22__season', 'sd_shotclock_24-22__season_type'],
                                right_on=['sd_shotclock_22-18__player', 'sd_shotclock_22-18__team',
                                            'sd_shotclock_22-18__season', 'sd_shotclock_22-18__season_type'],
                                how='left')

all_shotclock_play2 = pd.merge(all_shotclock_play1, shotclock_3_play,
                                left_on=['sd_shotclock_24-22__player', 'sd_shotclock_24-22__team',
                                            'sd_shotclock_24-22__season', 'sd_shotclock_24-22__season_type'],
                                right_on=['sd_shotclock_18-15__player', 'sd_shotclock_18-15__team',
                                            'sd_shotclock_18-15__season', 'sd_shotclock_18-15__season_type'],
                                how='left')

all_shotclock_play3 = pd.merge(all_shotclock_play2, shotclock_4_play,
                                left_on=['sd_shotclock_24-22__player', 'sd_shotclock_24-22__team',
                                            'sd_shotclock_24-22__season', 'sd_shotclock_24-22__season_type'],
                                right_on=['sd_shotclock_15-7__player', 'sd_shotclock_15-7__team',
                                            'sd_shotclock_15-7__season', 'sd_shotclock_15-7__season_type'],
                                how='left')

all_shotclock_play4 = pd.merge(all_shotclock_play3, shotclock_5_play,
                                left_on=['sd_shotclock_24-22__player', 'sd_shotclock_24-22__team',
                                            'sd_shotclock_24-22__season', 'sd_shotclock_24-22__season_type'],
                                right_on=['sd_shotclock_7-4__player', 'sd_shotclock_7-4__team',
                                            'sd_shotclock_7-4__season', 'sd_shotclock_7-4__season_type'],
                                how='left')

all_shotclock_play = pd.merge(all_shotclock_play4, shotclock_6_play,
                                left_on=['sd_shotclock_24-22__player', 'sd_shotclock_24-22__team',
                                            'sd_shotclock_24-22__season', 'sd_shotclock_24-22__season_type'],
                                right_on=['sd_shotclock_4-0__player', 'sd_shotclock_4-0__team',
                                            'sd_shotclock_4-0__season', 'sd_shotclock_4-0__season_type'],
                                how='left')

all_shotclock = pd.concat([all_shotclock_reg, all_shotclock_play], axis=0)
all_shotclock.to_csv('data/player/aggregates/ShotDash_All_Shotclock.csv', index=False)

## Dribbles

In [23]:
# check if dribbles/regular_season folder exists
if not os.path.exists('data/player/shot_dashboard/dribbles/regular_season/'):
    os.makedirs('data/player/shot_dashboard/dribbles/regular_season/')

# check if dribbles/playoffs folder exists
if not os.path.exists('data/player/shot_dashboard/dribbles/playoffs/'):
    os.makedirs('data/player/shot_dashboard/dribbles/playoffs/')

# move files to correct folder
dribbles_files = os.listdir('data/player/shot_dashboard/dribbles/')

for file in dribbles_files:
    if '.csv' in file:
        # move to correct folder
        if 'Regular' in file:
            shutil.move('data/player/shot_dashboard/dribbles/' + file, 'data/player/shot_dashboard/dribbles/regular_season/')
        else:
            shutil.move('data/player/shot_dashboard/dribbles/' + file, 'data/player/shot_dashboard/dribbles/playoffs/')
    

In [24]:
years = ['2021-22', '2020-21', '2019-20', '2018-19', '2017-18', '2016-17', '2015-16','2014-15', '2013-14']
season_types = ['Regular Season', 'Playoffs']
dribbles_urls = []
for year in years:
    for s_types in season_types:
        url = player_dribbles + '?SeasonYear=' + year + '&SeasonType=' + s_types + '&Season=' + year
        dribbles_urls.append(str(url))

In [25]:
drib_to_download = pd.DataFrame(dribbles_urls, columns = ['urls'])
# create new columns with option filenames
drib_to_download['filename1'] = drib_to_download.apply(lambda row: trans_urls(row['urls']), axis=1)
drib_to_download['filename_o1'] = drib_to_download['filename1'] + 'op_0-drib.csv'
drib_to_download['filename_o2'] = drib_to_download['filename1'] + 'op_1-drib.csv'
drib_to_download['filename_o3'] = drib_to_download['filename1'] + 'op_2-drib.csv'
drib_to_download['filename_o4'] = drib_to_download['filename1'] + 'op_3-6-drib.csv'
drib_to_download['filename_o5'] = drib_to_download['filename1'] + 'op_7-drib.csv'

# check to see if files exist already
folder1 = 'data/player/shot_dashboard/dribbles/playoffs/'
folder1_files = os.listdir(folder1)
folder2 = 'data/player/shot_dashboard/dribbles/regular_season/'
folder2_files = os.listdir(folder2)
files = folder1_files + folder2_files
files = [f for f in files if f.endswith('.csv')]
drib_to_download = drib_to_download[~drib_to_download['filename_o1'].isin(files)]
drib_to_download = drib_to_download[~drib_to_download['filename_o2'].isin(files)]
drib_to_download = drib_to_download[~drib_to_download['filename_o3'].isin(files)]
drib_to_download = drib_to_download[~drib_to_download['filename_o4'].isin(files)]
drib_to_download = drib_to_download[~drib_to_download['filename_o5'].isin(files)]

drib_to_download



,urls,filename1,filename_o1,filename_o2,filename_o3,filename_o4,filename_o5


In [26]:
if len(drib_to_download) > 0:
    to_dl = drib_to_download['urls'].tolist()
    get_data_w_double_header(to_dl, 'data/player/shot_dashboard/dribbles/', 5, ['0-drib', '1-drib', '2-drib', '3-6-drib', '7-drib'], 6)
else:
    print('no files to download')

no files to download


### Update this Year

In [27]:
url = player_dribbles + '?SeasonYear=2022-23&SeasonType=Regular Season&Season=2022-23'

driver = webdriver.Chrome()
get_data_w_double_header([url], 'data/player/shot_dashboard/dribbles/', 5, ['0-drib', '1-drib', '2-drib', '3-6-drib', '7-drib'], 6)

data/player/shot_dashboard/dribbles/shots-dribbles__2022-23_Regular 2022-23op_0-drib.csv Completed Successfully! 1 / 1 Complete!
data/player/shot_dashboard/dribbles/shots-dribbles__2022-23_Regular 2022-23op_1-drib.csv Completed Successfully! 2 / 1 Complete!
data/player/shot_dashboard/dribbles/shots-dribbles__2022-23_Regular 2022-23op_2-drib.csv Completed Successfully! 3 / 1 Complete!
data/player/shot_dashboard/dribbles/shots-dribbles__2022-23_Regular 2022-23op_3-6-drib.csv Completed Successfully! 4 / 1 Complete!
data/player/shot_dashboard/dribbles/shots-dribbles__2022-23_Regular 2022-23op_7-drib.csv Completed Successfully! 5 / 1 Complete!


In [28]:
# move files to correct folder
dribbles_files = os.listdir('data/player/shot_dashboard/dribbles/')

for file in dribbles_files:
    if '.csv' in file:
        # move to correct folder
        if 'Regular' in file:
            shutil.move('data/player/shot_dashboard/dribbles/' + file, 'data/player/shot_dashboard/dribbles/regular_season/')
        else:
            shutil.move('data/player/shot_dashboard/dribbles/' + file, 'data/player/shot_dashboard/dribbles/playoffs/')

#### Append

In [29]:
# append the data
drib_1_reg = append_the_data('data/player/shot_dashboard/dribbles/regular_season/', 'sd_drib_0_dribbles__', '0-drib')
drib_2_reg = append_the_data('data/player/shot_dashboard/dribbles/regular_season/', 'sd_drib_1_dribbles__', '1-drib')
drib_3_reg = append_the_data('data/player/shot_dashboard/dribbles/regular_season/', 'sd_drib_2_dribbles__', '2-drib')
drib_4_reg = append_the_data('data/player/shot_dashboard/dribbles/regular_season/', 'sd_drib_3-6_dribbles__', '3-6-drib')
drib_5_reg = append_the_data('data/player/shot_dashboard/dribbles/regular_season/', 'sd_drib_7_dribbles__', '7-drib')

In [30]:
drib_1_play = append_the_data('data/player/shot_dashboard/dribbles/playoffs/', 'sd_drib_0_dribbles__', '0-drib')
drib_2_play = append_the_data('data/player/shot_dashboard/dribbles/playoffs/', 'sd_drib_1_dribbles__', '1-drib')
drib_3_play = append_the_data('data/player/shot_dashboard/dribbles/playoffs/', 'sd_drib_2_dribbles__', '2-drib')
drib_4_play = append_the_data('data/player/shot_dashboard/dribbles/playoffs/', 'sd_drib_3-6_dribbles__', '3-6-drib')
drib_5_play = append_the_data('data/player/shot_dashboard/dribbles/playoffs/', 'sd_drib_7_dribbles__', '7-drib')

In [31]:
drib_1_reg.columns

Index(['sd_drib_0_dribbles__unnamed: 0', 'sd_drib_0_dribbles__player',
       'sd_drib_0_dribbles__team', 'sd_drib_0_dribbles__age',
       'sd_drib_0_dribbles__gp', 'sd_drib_0_dribbles__g',
       'sd_drib_0_dribbles__freq%', 'sd_drib_0_dribbles__fgm',
       'sd_drib_0_dribbles__fga', 'sd_drib_0_dribbles__fg%',
       'sd_drib_0_dribbles__efg%', 'sd_drib_0_dribbles__2fg freq%',
       'sd_drib_0_dribbles__2fgm', 'sd_drib_0_dribbles__2fga',
       'sd_drib_0_dribbles__2fg%', 'sd_drib_0_dribbles__3fg freq%',
       'sd_drib_0_dribbles__3pm', 'sd_drib_0_dribbles__3pa',
       'sd_drib_0_dribbles__3p%', 'sd_drib_0_dribbles__season',
       'sd_drib_0_dribbles__season_type'],
      dtype='object')

In [32]:
all_dribbles_reg1 = pd.merge(drib_1_reg, drib_2_reg,
                                left_on = ['sd_drib_0_dribbles__player', 'sd_drib_0_dribbles__team', 'sd_drib_0_dribbles__season', 'sd_drib_0_dribbles__season_type'],
                                right_on = ['sd_drib_1_dribbles__player', 'sd_drib_1_dribbles__team', 'sd_drib_1_dribbles__season', 'sd_drib_1_dribbles__season_type'],
                                how = 'left')

all_dribbles_reg1

,sd_drib_0_dribbles__unnamed: 0,sd_drib_0_dribbles__player,sd_drib_0_dribbles__team,sd_drib_0_dribbles__age,sd_drib_0_dribbles__gp,sd_drib_0_dribbles__g,sd_drib_0_dribbles__freq%,sd_drib_0_dribbles__fgm,sd_drib_0_dribbles__fga,sd_drib_0_dribbles__fg%,...,sd_drib_1_dribbles__2fg freq%,sd_drib_1_dribbles__2fgm,sd_drib_1_dribbles__2fga,sd_drib_1_dribbles__2fg%,sd_drib_1_dribbles__3fg freq%,sd_drib_1_dribbles__3pm,sd_drib_1_dribbles__3pa,sd_drib_1_dribbles__3p%,sd_drib_1_dribbles__season,sd_drib_1_dribbles__season_type
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013,Regular
1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013,Regular
2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013,Regular
3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013,Regular
4,2,Kevin Love,MIN,25.0,77.0,77.0,62.0,5.5,10.9,50.5,...,32.9,3.5,5.8,60.3,29.1,2.0,5.1,39.3,2013,Regular
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4163,477,Thanasis Antetokounmpo,MIL,30.0,16.0,4.0,83.3,0.1,0.3,40.0,...,66.7,0.1,0.3,50.0,16.7,0.0,0.1,0.0,2022,Regular
4164,478,Leandro Bolmaro,UTA,22.0,10.0,3.0,20.0,0.0,0.3,0.0,...,0.0,0.0,0.0,-,20.0,0.0,0.3,0.0,2022,Regular
4165,479,Jaden Springer,PHI,20.0,4.0,1.0,20.0,0.3,0.3,100.0,...,20.0,0.3,0.3,100,0.0,0.0,0.0,-,2022,Regular
4166,480,Alize Johnson,SAS,26.0,4.0,1.0,16.7,0.0,0.3,0.0,...,0.0,0.0,0.0,-,16.7,0.0,0.3,0.0,2022,Regular


In [33]:
all_dribbles_reg2 = pd.merge(all_dribbles_reg1, drib_3_reg,
                                left_on = ['sd_drib_0_dribbles__player', 'sd_drib_0_dribbles__team', 'sd_drib_0_dribbles__season', 'sd_drib_0_dribbles__season_type'],
                                right_on = ['sd_drib_2_dribbles__player', 'sd_drib_2_dribbles__team', 'sd_drib_2_dribbles__season', 'sd_drib_2_dribbles__season_type'],
                                how = 'left')

all_dribbles_reg3 = pd.merge(all_dribbles_reg2, drib_4_reg,
                                left_on = ['sd_drib_0_dribbles__player', 'sd_drib_0_dribbles__team', 'sd_drib_0_dribbles__season', 'sd_drib_0_dribbles__season_type'],
                                right_on = ['sd_drib_3-6_dribbles__player', 'sd_drib_3-6_dribbles__team', 'sd_drib_3-6_dribbles__season', 'sd_drib_3-6_dribbles__season_type'],
                                how = 'left')

all_dribbles_reg4 = pd.merge(all_dribbles_reg3, drib_5_reg,
                                left_on = ['sd_drib_0_dribbles__player', 'sd_drib_0_dribbles__team', 'sd_drib_0_dribbles__season', 'sd_drib_0_dribbles__season_type'],
                                right_on = ['sd_drib_7_dribbles__player', 'sd_drib_7_dribbles__team', 'sd_drib_7_dribbles__season', 'sd_drib_7_dribbles__season_type'],
                                how = 'left')

all_dribbles_reg4

,sd_drib_0_dribbles__unnamed: 0,sd_drib_0_dribbles__player,sd_drib_0_dribbles__team,sd_drib_0_dribbles__age,sd_drib_0_dribbles__gp,sd_drib_0_dribbles__g,sd_drib_0_dribbles__freq%,sd_drib_0_dribbles__fgm,sd_drib_0_dribbles__fga,sd_drib_0_dribbles__fg%,...,sd_drib_7_dribbles__2fg freq%,sd_drib_7_dribbles__2fgm,sd_drib_7_dribbles__2fga,sd_drib_7_dribbles__2fg%,sd_drib_7_dribbles__3fg freq%,sd_drib_7_dribbles__3pm,sd_drib_7_dribbles__3pa,sd_drib_7_dribbles__3p%,sd_drib_7_dribbles__season,sd_drib_7_dribbles__season_type
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013,Regular
1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013,Regular
2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013,Regular
3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013,Regular
4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013,Regular
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4387,477,Thanasis Antetokounmpo,MIL,30.0,16.0,4.0,83.3,0.1,0.3,40.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4388,478,Leandro Bolmaro,UTA,22.0,10.0,3.0,20.0,0.0,0.3,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4389,479,Jaden Springer,PHI,20.0,4.0,1.0,20.0,0.3,0.3,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4390,480,Alize Johnson,SAS,26.0,4.0,1.0,16.7,0.0,0.3,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
# drop nan in player column
all_dribbles_reg4 = all_dribbles_reg4.dropna(subset = ['sd_drib_0_dribbles__player'])
all_dribbles_reg4

,sd_drib_0_dribbles__unnamed: 0,sd_drib_0_dribbles__player,sd_drib_0_dribbles__team,sd_drib_0_dribbles__age,sd_drib_0_dribbles__gp,sd_drib_0_dribbles__g,sd_drib_0_dribbles__freq%,sd_drib_0_dribbles__fgm,sd_drib_0_dribbles__fga,sd_drib_0_dribbles__fg%,...,sd_drib_7_dribbles__2fg freq%,sd_drib_7_dribbles__2fgm,sd_drib_7_dribbles__2fga,sd_drib_7_dribbles__2fg%,sd_drib_7_dribbles__3fg freq%,sd_drib_7_dribbles__3pm,sd_drib_7_dribbles__3pa,sd_drib_7_dribbles__3p%,sd_drib_7_dribbles__season,sd_drib_7_dribbles__season_type
32,2,Kevin Love,MIN,25.0,77.0,77.0,62.0,5.5,10.9,50.5,...,32.9,3.5,5.8,60.3,29.1,2.0,5.1,39.3,2013,Regular
33,3,Ryan Anderson,NOP,26.0,22.0,22.0,68.9,4.8,10.7,44.7,...,23.5,1.8,3.6,50.0,45.5,3.0,7.0,41.9,2013,Regular
34,4,Al Horford,ATL,28.0,29.0,29.0,74.6,6.5,10.4,61.7,...,72.4,6.3,10.1,62.2,2.2,0.1,0.3,44.4,2013,Regular
35,5,LaMarcus Aldridge,POR,28.0,69.0,69.0,50.5,5.3,10.1,52.3,...,49.9,5.3,10.0,52.7,0.7,0.0,0.1,22.2,2013,Regular
36,6,Serge Ibaka,OKC,24.0,81.0,81.0,83.3,5.5,9.7,56.9,...,77.0,5.3,9.0,58.5,6.2,0.3,0.7,37.3,2013,Regular
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4387,477,Thanasis Antetokounmpo,MIL,30.0,16.0,4.0,83.3,0.1,0.3,40.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4388,478,Leandro Bolmaro,UTA,22.0,10.0,3.0,20.0,0.0,0.3,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4389,479,Jaden Springer,PHI,20.0,4.0,1.0,20.0,0.3,0.3,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4390,480,Alize Johnson,SAS,26.0,4.0,1.0,16.7,0.0,0.3,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
all_dribbles_reg4.to_csv('data/player/aggregates/ShotDash_Dribbles_Regular_Season.csv', index = False)

In [36]:
all_dribbles_play1 = pd.merge(drib_1_play, drib_2_play,
                                left_on = ['sd_drib_0_dribbles__player', 'sd_drib_0_dribbles__team', 'sd_drib_0_dribbles__season', 'sd_drib_0_dribbles__season_type'],
                                right_on = ['sd_drib_1_dribbles__player', 'sd_drib_1_dribbles__team', 'sd_drib_1_dribbles__season', 'sd_drib_1_dribbles__season_type'],
                                how = 'left')

all_dribbles_play2 = pd.merge(all_dribbles_play1, drib_3_play,
                                left_on = ['sd_drib_0_dribbles__player', 'sd_drib_0_dribbles__team', 'sd_drib_0_dribbles__season', 'sd_drib_0_dribbles__season_type'],
                                right_on = ['sd_drib_2_dribbles__player', 'sd_drib_2_dribbles__team', 'sd_drib_2_dribbles__season', 'sd_drib_2_dribbles__season_type'],
                                how = 'left')

all_dribbles_play3 = pd.merge(all_dribbles_play2, drib_4_play,
                                left_on = ['sd_drib_0_dribbles__player', 'sd_drib_0_dribbles__team', 'sd_drib_0_dribbles__season', 'sd_drib_0_dribbles__season_type'],
                                right_on = ['sd_drib_3-6_dribbles__player', 'sd_drib_3-6_dribbles__team', 'sd_drib_3-6_dribbles__season', 'sd_drib_3-6_dribbles__season_type'],
                                how = 'left')

all_dribbles_play4 = pd.merge(all_dribbles_play3, drib_5_play,
                                left_on = ['sd_drib_0_dribbles__player', 'sd_drib_0_dribbles__team', 'sd_drib_0_dribbles__season', 'sd_drib_0_dribbles__season_type'],
                                right_on = ['sd_drib_7_dribbles__player', 'sd_drib_7_dribbles__team', 'sd_drib_7_dribbles__season', 'sd_drib_7_dribbles__season_type'],
                                how = 'left')

all_dribbles_play4

,sd_drib_0_dribbles__unnamed: 0,sd_drib_0_dribbles__player,sd_drib_0_dribbles__team,sd_drib_0_dribbles__age,sd_drib_0_dribbles__gp,sd_drib_0_dribbles__g,sd_drib_0_dribbles__freq%,sd_drib_0_dribbles__fgm,sd_drib_0_dribbles__fga,sd_drib_0_dribbles__fg%,...,sd_drib_7_dribbles__2fg freq%,sd_drib_7_dribbles__2fgm,sd_drib_7_dribbles__2fga,sd_drib_7_dribbles__2fg%,sd_drib_7_dribbles__3fg freq%,sd_drib_7_dribbles__3pm,sd_drib_7_dribbles__3pa,sd_drib_7_dribbles__3p%,sd_drib_7_dribbles__season,sd_drib_7_dribbles__season_type
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013,Playoffs
1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013,Playoffs
2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013,Playoffs
3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013,Playoffs
4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013,Playoffs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1842,200,Facundo Campazzo,DEN,31.0,4.0,1.0,50.0,0.0,0.3,0.0,...,0.0,0.0,0.0,-,50.0,0.0,0.3,0.0,2021,Playoffs
1843,201,Malik Fitts,BOS,24.0,9.0,2.0,50.0,0.2,0.2,100.0,...,0.0,0.0,0.0,-,50.0,0.2,0.2,100,2021,Playoffs
1844,202,Jaden Springer,PHI,19.0,5.0,1.0,16.7,0.0,0.2,0.0,...,0.0,0.0,0.0,-,16.7,0.0,0.2,0.0,2021,Playoffs
1845,203,Isaiah Joe,PHI,22.0,7.0,1.0,20.0,0.0,0.1,0.0,...,0.0,0.0,0.0,-,20.0,0.0,0.1,0.0,2021,Playoffs


In [37]:
# drop nan in player column
all_dribbles_play4 = all_dribbles_play4.dropna(subset = ['sd_drib_0_dribbles__player'])
all_dribbles_play4

,sd_drib_0_dribbles__unnamed: 0,sd_drib_0_dribbles__player,sd_drib_0_dribbles__team,sd_drib_0_dribbles__age,sd_drib_0_dribbles__gp,sd_drib_0_dribbles__g,sd_drib_0_dribbles__freq%,sd_drib_0_dribbles__fgm,sd_drib_0_dribbles__fga,sd_drib_0_dribbles__fg%,...,sd_drib_7_dribbles__2fg freq%,sd_drib_7_dribbles__2fgm,sd_drib_7_dribbles__2fga,sd_drib_7_dribbles__2fg%,sd_drib_7_dribbles__3fg freq%,sd_drib_7_dribbles__3pm,sd_drib_7_dribbles__3pa,sd_drib_7_dribbles__3p%,sd_drib_7_dribbles__season,sd_drib_7_dribbles__season_type
32,2,Al Jefferson,CHA,29.0,3.0,3.0,66.7,6.0,11.3,52.9,...,66.7,6.0,11.3,52.9,0.0,0.0,0.0,-,2013,Playoffs
33,3,LaMarcus Aldridge,POR,28.0,11.0,11.0,43.2,4.8,9.6,50.5,...,42.8,4.7,9.4,50.0,0.4,0.1,0.1,100,2013,Playoffs
34,4,Marc Gasol,MEM,29.0,7.0,7.0,56.5,4.0,8.7,45.9,...,56.5,4.0,8.7,45.9,0.0,0.0,0.0,-,2013,Playoffs
35,5,Kevin Durant,OKC,25.0,19.0,19.0,38.3,4.3,8.3,51.3,...,18.0,2.5,3.9,63.5,20.4,1.8,4.4,40.5,2013,Playoffs
36,6,Nene,WAS,31.0,10.0,10.0,63.8,4.0,8.3,48.2,...,63.8,4.0,8.3,48.2,0.0,0.0,0.0,-,2013,Playoffs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1842,200,Facundo Campazzo,DEN,31.0,4.0,1.0,50.0,0.0,0.3,0.0,...,0.0,0.0,0.0,-,50.0,0.0,0.3,0.0,2021,Playoffs
1843,201,Malik Fitts,BOS,24.0,9.0,2.0,50.0,0.2,0.2,100.0,...,0.0,0.0,0.0,-,50.0,0.2,0.2,100,2021,Playoffs
1844,202,Jaden Springer,PHI,19.0,5.0,1.0,16.7,0.0,0.2,0.0,...,0.0,0.0,0.0,-,16.7,0.0,0.2,0.0,2021,Playoffs
1845,203,Isaiah Joe,PHI,22.0,7.0,1.0,20.0,0.0,0.1,0.0,...,0.0,0.0,0.0,-,20.0,0.0,0.1,0.0,2021,Playoffs


In [38]:
all_dribbles_reg4.to_csv('data/player/aggregates/ShotDash_Dribbles_Playoffs.csv', index = False)

In [39]:
# concat all dribbles
all_dribbles = pd.concat([all_dribbles_reg4, all_dribbles_play4])
all_dribbles.to_csv('data/player/aggregates/ShotDash_All_Dribbles.csv', index = False)

## TouchTime

In [40]:
# Check if touchtime/regular_season folder exists
if not os.path.exists('data/player/shot_dashboard/touch_time/regular_season/'):
    os.makedirs('data/player/shot_dashboard/touch_time/regular_season/')

# Check if touchtime/playoffs folder exists
if not os.path.exists('data/player/shot_dashboard/touch_time/playoffs/'):
    os.makedirs('data/player/shot_dashboard/touch_time/playoffs/')

    # move files to appropriate folder
for file in os.listdir('data/player/shot_dashboard/touch_time/'):
    if '.csv' in file:
        if 'Regular' in file:
            shutil.move('data/player/shot_dashboard/touch_time/' + file, 'data/player/shot_dashboard/touch_time/regular_season/' + file)
        else:
            shutil.move('data/player/shot_dashboard/touch_time/' + file, 'data/player/shot_dashboard/touch_time/playoffs/' + file)



In [41]:
touchtime_urls = []
for year in years:
    for s_types in season_types:
        url = player_touchtime + '?SeasonYear=' + year + '&SeasonType=' + s_types + '&Season=' + year
        touchtime_urls.append(str(url))

In [42]:
touchtime_to_download = pd.DataFrame(touchtime_urls, columns = ['url'])

# Create new columns with option filenames
touchtime_to_download['filename1'] = touchtime_to_download.apply(lambda row: trans_urls(row['url']), axis = 1)
touchtime_to_download['filename_o1'] = touchtime_to_download['filename1'] +  'op_0-2.csv'
touchtime_to_download['filename_o2'] = touchtime_to_download['filename1'] +  'op_2-6.csv'
touchtime_to_download['filename_o3'] = touchtime_to_download['filename1'] +  'op_6+ seconds.csv'

# check to see if files already exist
folder1 = 'data/player/shot_dashboard/touch_time/regular_season/'
folder2 = 'data/player/shot_dashboard/touch_time/playoffs/'
folder1_files = os.listdir(folder1)
folder2_files = os.listdir(folder2)
files = folder1_files + folder2_files
files = [file for file in files if '.csv' in file]

touchtime_to_download = touchtime_to_download[~touchtime_to_download['filename_o1'].isin(files)]
touchtime_to_download = touchtime_to_download[~touchtime_to_download['filename_o2'].isin(files)]
touchtime_to_download = touchtime_to_download[~touchtime_to_download['filename_o3'].isin(files)]

touchtime_to_download


,url,filename1,filename_o1,filename_o2,filename_o3


In [43]:
if len(touchtime_to_download) > 0:
    to_dl = touchtime_to_download['url'].tolist()
    get_data_w_double_header(to_dl, 'data/player/shot_dashboard/touch_time/', 3, ['0-2', '2-6', '6+ seconds'], 6)
else:
    print('All files already downloaded')

All files already downloaded


### Update This Year

In [45]:
url = player_touchtime + '?SeasonYear=' + '2022-23' + '&SeasonType=Regular Season'  + '&Season=' + '2022-23'

driver = webdriver.Chrome()
get_data_w_double_header([url], 'data/player/shot_dashboard/touch_time/', 3, ['0-2', '2-6', '6+ seconds'], 6)


data/player/shot_dashboard/touch_time/shots-touch-time__2022-23_Regular 2022-23op_0-2.csv Completed Successfully! 1 / 1 Complete!
data/player/shot_dashboard/touch_time/shots-touch-time__2022-23_Regular 2022-23op_2-6.csv Completed Successfully! 2 / 1 Complete!
data/player/shot_dashboard/touch_time/shots-touch-time__2022-23_Regular 2022-23op_6+ seconds.csv Completed Successfully! 3 / 1 Complete!


In [46]:
# check if regular season or playoffs folders exist
if not os.path.exists('data/player/shot_dashboard/touch_time/regular_season/'):
    os.makedirs('data/player/shot_dashboard/touch_time/regular_season/')
if not os.path.exists('data/player/shot_dashboard/touch_time/playoffs/'):
    os.makedirs('data/player/shot_dashboard/touch_time/playoffs/')

# move files to regular season or playoffs folders
for file in os.listdir('data/player/shot_dashboard/touch_time/'):
    if '.csv' in file:
        if 'Regular' in file:
            shutil.move('data/player/shot_dashboard/touch_time/' + file, 'data/player/shot_dashboard/touch_time/regular_season/' + file)
        elif 'Playoffs' in file:
            shutil.move('data/player/shot_dashboard/touch_time/' + file, 'data/player/shot_dashboard/touch_time/playoffs/' + file)
            

### Append

In [47]:
touchtime_1_reg = append_the_data('data/player/shot_dashboard/touch_time/regular_season/', 'sd_touchtime_0_2_sec_', '0-2')
touchtime_2_reg = append_the_data('data/player/shot_dashboard/touch_time/regular_season/', 'sd_touchtime_2_6_sec_', '2-6')
touchtime_3_reg = append_the_data('data/player/shot_dashboard/touch_time/regular_season/', 'sd_touchtime_6_sec_', '6+')

touchtime_1_play = append_the_data('data/player/shot_dashboard/touch_time/playoffs/', 'sd_touchtime_0_2_sec_', '0-2')
touchtime_2_play = append_the_data('data/player/shot_dashboard/touch_time/playoffs/', 'sd_touchtime_2_6_sec_', '2-6')
touchtime_3_play = append_the_data('data/player/shot_dashboard/touch_time/playoffs/', 'sd_touchtime_6_sec_', '6+')


In [48]:
touchtime_1_reg.columns

Index(['sd_touchtime_0_2_sec_unnamed: 0', 'sd_touchtime_0_2_sec_player',
       'sd_touchtime_0_2_sec_team', 'sd_touchtime_0_2_sec_age',
       'sd_touchtime_0_2_sec_gp', 'sd_touchtime_0_2_sec_g',
       'sd_touchtime_0_2_sec_freq%', 'sd_touchtime_0_2_sec_fgm',
       'sd_touchtime_0_2_sec_fga', 'sd_touchtime_0_2_sec_fg%',
       'sd_touchtime_0_2_sec_efg%', 'sd_touchtime_0_2_sec_2fg freq%',
       'sd_touchtime_0_2_sec_2fgm', 'sd_touchtime_0_2_sec_2fga',
       'sd_touchtime_0_2_sec_2fg%', 'sd_touchtime_0_2_sec_3fg freq%',
       'sd_touchtime_0_2_sec_3pm', 'sd_touchtime_0_2_sec_3pa',
       'sd_touchtime_0_2_sec_3p%', 'sd_touchtime_0_2_sec_season',
       'sd_touchtime_0_2_sec_season_type'],
      dtype='object')

In [49]:
all_touchtime_reg1 = pd.merge(touchtime_1_reg, touchtime_2_reg,
                                left_on= ['sd_touchtime_0_2_sec_player', 'sd_touchtime_0_2_sec_team', 
                                            'sd_touchtime_0_2_sec_season', 'sd_touchtime_0_2_sec_season_type'],
                                right_on= ['sd_touchtime_2_6_sec_player', 'sd_touchtime_2_6_sec_team',
                                            'sd_touchtime_2_6_sec_season', 'sd_touchtime_2_6_sec_season_type'],
                                how = 'left')

all_touchtime_reg2 = pd.merge(all_touchtime_reg1, touchtime_3_reg,
                                left_on= ['sd_touchtime_0_2_sec_player', 'sd_touchtime_0_2_sec_team',
                                            'sd_touchtime_0_2_sec_season', 'sd_touchtime_0_2_sec_season_type'],
                                right_on= ['sd_touchtime_6_sec_player', 'sd_touchtime_6_sec_team',
                                            'sd_touchtime_6_sec_season', 'sd_touchtime_6_sec_season_type'],
                                how = 'left')

all_touchtime_reg2.to_csv('data/player/aggregates/ShotDash_Touchtime_Regular_Season.csv', index = False)

In [50]:
all_touchtime_play1 = pd.merge(touchtime_1_play, touchtime_2_play,
                                left_on= ['sd_touchtime_0_2_sec_player', 'sd_touchtime_0_2_sec_team',
                                            'sd_touchtime_0_2_sec_season', 'sd_touchtime_0_2_sec_season_type'],
                                right_on= ['sd_touchtime_2_6_sec_player', 'sd_touchtime_2_6_sec_team',
                                            'sd_touchtime_2_6_sec_season', 'sd_touchtime_2_6_sec_season_type'],
                                how = 'left')

all_touchtime_play2 = pd.merge(all_touchtime_play1, touchtime_3_play,
                                left_on= ['sd_touchtime_0_2_sec_player', 'sd_touchtime_0_2_sec_team',
                                            'sd_touchtime_0_2_sec_season', 'sd_touchtime_0_2_sec_season_type'],
                                right_on= ['sd_touchtime_6_sec_player', 'sd_touchtime_6_sec_team',
                                            'sd_touchtime_6_sec_season', 'sd_touchtime_6_sec_season_type'],
                                how = 'left')

all_touchtime_play2.to_csv('data/player/aggregates/ShotDash_Touchtime_Playoffs.csv', index = False)


In [51]:
# concat all touchtime data
all_touchtime = pd.concat([all_touchtime_reg2, all_touchtime_play2], axis = 0)
all_touchtime.to_csv('data/player/aggregates/ShotDash_Touchtime_All.csv', index = False)

## Closest Defenders

In [52]:
if not os.path.exists('data/player/shot_dashboard/closest_defender/regular_season/'):
    os.makedirs('data/player/shot_dashboard/closest_defender/regular_season/')

if not os.path.exists('data/player/shot_dashboard/closest_defender/playoffs/'):
    os.makedirs('data/player/shot_dashboard/closest_defender/playoffs/')

# move files to regular season or playoffs folders
for file in os.listdir('data/player/shot_dashboard/closest_defender/'):
    if '.csv' in file:
        if 'Regular' in file:
            shutil.move('data/player/shot_dashboard/closest_defender/' + file, 'data/player/shot_dashboard/closest_defender/regular_season/' + file)
        elif 'Playoffs' in file:
            shutil.move('data/player/shot_dashboard/closest_defender/' + file, 'data/player/shot_dashboard/closest_defender/playoffs/' + file)

In [53]:
closest_defender_urls = []
for year in years:
    for s_types in season_types:
        url = player_closest_defender + '?SeasonYear=' + year + '&SeasonType=' + s_types + '&Season=' + year
        closest_defender_urls.append(str(url))

In [54]:
closest_def_to_download = pd.DataFrame(closest_defender_urls, columns = ['url'])

# Create new columns with option filenames
closest_def_to_download['filename1'] = closest_def_to_download.apply(lambda row: trans_urls(row['url']), axis = 1)
closest_def_to_download['filename_1'] = closest_def_to_download['filename1'] + 'op_0-2.csv'
closest_def_to_download['filename_2'] = closest_def_to_download['filename1'] + 'op_2-4.csv'
closest_def_to_download['filename_3'] = closest_def_to_download['filename1'] + 'op_4-6.csv'
closest_def_to_download['filename_4'] = closest_def_to_download['filename1'] + 'op_6+ feet.csv'

# Check to see if files already exist
folder1 = 'data/player/shot_dashboard/closest_defender/regular_season/'
folder2 = 'data/player/shot_dashboard/closest_defender/playoffs/'
folder1_files = os.listdir(folder1)
folder2_files = os.listdir(folder2)
files = folder1_files + folder2_files
files = [x for x in files if '.csv' in x]

closest_def_to_download = closest_def_to_download[~closest_def_to_download['filename_1'].isin(files)]
closest_def_to_download = closest_def_to_download[~closest_def_to_download['filename_2'].isin(files)]
closest_def_to_download = closest_def_to_download[~closest_def_to_download['filename_3'].isin(files)]
closest_def_to_download = closest_def_to_download[~closest_def_to_download['filename_4'].isin(files)]

closest_def_to_download

,url,filename1,filename_1,filename_2,filename_3,filename_4


In [55]:
if len(closest_def_to_download) > 0:
    to_dl = closest_def_to_download['url'].tolist()
    get_data_w_double_header(closest_defender_urls, 'data/player/shot_dashboard/closest_defender/', 4, ['0-2', '2-4', '4-6', '6+ feet'], 6)
else:
    print('All files already downloaded')

All files already downloaded


### Update This Year

In [57]:
url = player_closest_defender + '?SeasonYear=2022-23&SeasonType=Regular+Season&Season=2022-23'

driver = webdriver.Chrome()
get_data_w_double_header([url], 'data/player/shot_dashboard/closest_defender/', 4, ['0-2', '2-4', '4-6', '6+ feet'], 6)



data/player/shot_dashboard/closest_defender/shots-closest-defender__2022-23_Regular+2022-23op_0-2.csv Completed Successfully! 1 / 1 Complete!
data/player/shot_dashboard/closest_defender/shots-closest-defender__2022-23_Regular+2022-23op_2-4.csv Completed Successfully! 2 / 1 Complete!
data/player/shot_dashboard/closest_defender/shots-closest-defender__2022-23_Regular+2022-23op_4-6.csv Completed Successfully! 3 / 1 Complete!
data/player/shot_dashboard/closest_defender/shots-closest-defender__2022-23_Regular+2022-23op_6+ feet.csv Completed Successfully! 4 / 1 Complete!


In [58]:
# move files to regular season or playoffs folders
for file in os.listdir('data/player/shot_dashboard/closest_defender/'):
    if '.csv' in file:
        if 'Regular' in file:
            shutil.move('data/player/shot_dashboard/closest_defender/' + file, 'data/player/shot_dashboard/closest_defender/regular_season/' + file)
        elif 'Playoffs' in file:
            shutil.move('data/player/shot_dashboard/closest_defender/' + file, 'data/player/shot_dashboard/closest_defender/playoffs/' + file)

### Append

In [59]:
closest_def_1_reg = append_the_data('data/player/shot_dashboard/closest_defender/regular_season/', 'sd_closest_D_0-2__', '0-2')
closest_def_2_reg = append_the_data('data/player/shot_dashboard/closest_defender/regular_season/', 'sd_closest_D_2-4__', '2-4')
closest_def_3_reg = append_the_data('data/player/shot_dashboard/closest_defender/regular_season/', 'sd_closest_D_4-6__', '4-6')
closest_def_4_reg = append_the_data('data/player/shot_dashboard/closest_defender/regular_season/', 'sd_closest_D_6+__', '6+')

closest_def_1_play = append_the_data('data/player/shot_dashboard/closest_defender/playoffs/', 'sd_closest_D_0-2__', '0-2')
closest_def_2_play = append_the_data('data/player/shot_dashboard/closest_defender/playoffs/', 'sd_closest_D_2-4__', '2-4')
closest_def_3_play = append_the_data('data/player/shot_dashboard/closest_defender/playoffs/', 'sd_closest_D_4-6__', '4-6')
closest_def_4_play = append_the_data('data/player/shot_dashboard/closest_defender/playoffs/', 'sd_closest_D_6+__', '6+')


In [60]:
closest_def_2_reg

,sd_closest_d_2-4__unnamed: 0,sd_closest_d_2-4__player,sd_closest_d_2-4__team,sd_closest_d_2-4__age,sd_closest_d_2-4__gp,sd_closest_d_2-4__g,sd_closest_d_2-4__freq%,sd_closest_d_2-4__fgm,sd_closest_d_2-4__fga,sd_closest_d_2-4__fg%,...,sd_closest_d_2-4__2fg freq%,sd_closest_d_2-4__2fgm,sd_closest_d_2-4__2fga,sd_closest_d_2-4__2fg%,sd_closest_d_2-4__3fg freq%,sd_closest_d_2-4__3pm,sd_closest_d_2-4__3pa,sd_closest_d_2-4__3p%,sd_closest_d_2-4__season,sd_closest_d_2-4__season_type
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013,Regular
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013,Regular
2,2,Nikola Pekovic,MIN,28.0,54.0,52.0,46.3,2.9,5.4,53.8,...,46.3,2.9,5.4,53.8,0.0,0.0,0.0,-,2013,Regular
3,3,David Lee,GSW,31.0,68.0,68.0,37.5,2.7,4.8,56.3,...,37.5,2.7,4.8,56.3,0.0,0.0,0.0,-,2013,Regular
4,4,Zach Randolph,MEM,32.0,79.0,77.0,34.2,2.6,4.7,55.4,...,34.2,2.6,4.7,55.4,0.0,0.0,0.0,-,2013,Regular
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,427,Ochai Agbaji,UTA,22.0,16.0,1.0,2.4,0.0,0.1,0.0,...,2.4,0.0,0.1,0.0,0.0,0.0,0.0,-,2022,Regular
428,428,Isaiah Livers,DET,24.0,22.0,1.0,1.0,0.1,0.1,100.0,...,1.0,0.1,0.1,100,0.0,0.0,0.0,-,2022,Regular
429,429,Jake LaRavia,MEM,21.0,20.0,1.0,1.4,0.0,0.1,0.0,...,1.4,0.0,0.1,0.0,0.0,0.0,0.0,-,2022,Regular
430,430,Anthony Lamb,GSW,24.0,26.0,1.0,0.9,0.0,0.0,100.0,...,0.9,0.0,0.0,100,0.0,0.0,0.0,-,2022,Regular


In [53]:
all_closest_reg1 = pd.merge(closest_def_1_reg, closest_def_2_reg,
                            left_on= ['sd_closest_d_0-2__player', 'sd_closest_d_0-2__team', 
                                            'sd_closest_d_0-2__season', 'sd_closest_d_0-2__season_type'],
                            right_on= ['sd_closest_d_2-4__player', 'sd_closest_d_2-4__team',
                                            'sd_closest_d_2-4__season', 'sd_closest_d_2-4__season_type'],
                            how = 'left')

all_closest_reg2 = pd.merge(all_closest_reg1, closest_def_3_reg,
                            left_on= ['sd_closest_d_0-2__player', 'sd_closest_d_0-2__team',
                                            'sd_closest_d_0-2__season', 'sd_closest_d_0-2__season_type'],
                            right_on= ['sd_closest_d_4-6__player', 'sd_closest_d_4-6__team',
                                            'sd_closest_d_4-6__season', 'sd_closest_d_4-6__season_type'],
                            how = 'left')

all_closest_reg3 = pd.merge(all_closest_reg2, closest_def_4_reg,
                            left_on= ['sd_closest_d_0-2__player', 'sd_closest_d_0-2__team', 
                                            'sd_closest_d_0-2__season', 'sd_closest_d_0-2__season_type'],
                            right_on= ['sd_closest_d_6+__player', 'sd_closest_d_6+__team',
                                            'sd_closest_d_6+__season', 'sd_closest_d_6+__season_type'],
                            how = 'left')



In [54]:
all_closest_def_play1 = pd.merge(closest_def_1_play, closest_def_2_play,
                            left_on= ['sd_closest_d_0-2__player', 'sd_closest_d_0-2__team',
                                            'sd_closest_d_0-2__season', 'sd_closest_d_0-2__season_type'],
                            right_on= ['sd_closest_d_2-4__player', 'sd_closest_d_2-4__team',
                                            'sd_closest_d_2-4__season', 'sd_closest_d_2-4__season_type'],
                            how = 'left')

all_closest_def_play2 = pd.merge(all_closest_def_play1, closest_def_3_play,
                            left_on= ['sd_closest_d_0-2__player', 'sd_closest_d_0-2__team',
                                            'sd_closest_d_0-2__season', 'sd_closest_d_0-2__season_type'],
                            right_on= ['sd_closest_d_4-6__player', 'sd_closest_d_4-6__team',
                                            'sd_closest_d_4-6__season', 'sd_closest_d_4-6__season_type'],
                            how = 'left')

all_closest_def_play3 = pd.merge(all_closest_def_play2, closest_def_4_play,
                            left_on= ['sd_closest_d_0-2__player', 'sd_closest_d_0-2__team',
                                            'sd_closest_d_0-2__season', 'sd_closest_d_0-2__season_type'],
                            right_on= ['sd_closest_d_6+__player', 'sd_closest_d_6+__team',
                                            'sd_closest_d_6+__season', 'sd_closest_d_6+__season_type'],

                            how = 'left')


In [ ]:
# Concat all closest d data
all_closest_def = pd.concat([all_closest_reg3, all_closest_def_play3], axis=0)
all_closest_def.to_csv('data/player/aggregates/ShotDash_All_Closest_Defender.csv', index=False)